In [1]:
import pandas as pd
from datasets import load_dataset

In [2]:
metadata_raw = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Books", trust_remote_code=True)

Loading dataset shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [3]:
metadata_raw_df = metadata_raw['full'].to_pandas()

In [4]:
metadata_raw_df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Books,Chaucer,4.5,29,[],[],8.23,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Peter Ackroyd (Author),"[Books, Literature & Fiction, History & Critic...","{""Publisher"": ""Chatto & Windus; First Edition ...",0701169850,None,"Hardcover – Import, January 1, 2004",{'avatar': 'https://m.media-amazon.com/images/...
1,Books,Notes from a Kidwatcher,5.0,1,[Contains 23 selected articles by this influen...,"[About the Author, SANDRA WILDE, Ph.D., is wid...",3.52,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Sandra Wilde (Editor),"[Books, Reference, Words, Language & Grammar]","{""Publisher"": ""Heinemann; First Edition (May 2...",0435088688,None,First Edition,{'avatar': 'https://m.media-amazon.com/images/...
2,Books,Service: A Navy SEAL at War,4.7,3421,"[Marcus Luttrell, author of the #1 bestseller,...","[Review, Praise for SERVICE""An action-packed.....",17.17,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}","Marcus Luttrell (Author), James D. Hornfischer","[Books, Biographies & Memoirs, Leaders & Notab...","{""Publisher"": ""Little, Brown and Company; 1st ...",0316185361,None,"Hardcover – May 8, 2012",{'avatar': 'https://m.media-amazon.com/images/...
3,Books,Monstrous Stories #4: The Day the Mice Stood S...,4.4,40,"[Funny, light-hearted monster stories that are...",[],7.43,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Dr. Roach (Author),"[Books, Children's Books, Science Fiction & Fa...","{""Publisher"": ""Scholastic Paperbacks; Reprint ...",0545425573,None,"Paperback – October 29, 2013",None
4,Buy a Kindle,Parker & Knight,4.5,381,"[From REMINGTON KANE, the author of The Taken!...",[],0.0,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Remington Kane (Author) Format: Kindle Edition,"[Books, Mystery, Thriller & Suspense, Thriller...","{""Publication date"": ""May 18, 2014"", ""Language...",B00KFOP3RG,None,Kindle Edition,{'avatar': 'https://m.media-amazon.com/images/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448176,Books,Please Excuse My Daughter,4.3,69,"[Look out for Julie's new book,, The Almost Le...","[About the Author, Julie Klam grew up in Bedfo...",36.06,"{'hi_res': [], 'large': [], 'thumb': [], 'vari...","{'title': [], 'url': [], 'user_id': []}",Julie Klam (Author),"[Books, Biographies & Memoirs, Community & Cul...","{""Publisher"": ""Riverhead Books; Reprint editio...",1594483574,None,"Paperback – April 7, 2009",{'avatar': 'https://m.media-amazon.com/images/...
4448177,Books,Inside the Southeast Asian Kitchen: Foodlore a...,5.0,1,[A sumptuous gastronomic tour of ten Southeast...,[],75.0,"{'hi_res': [], 'large': [], 'thumb': [], 'vari...","{'title': [], 'url': [], 'user_id': []}",Artpostasia Pte Ltd. (Author),"[Books, Cookbooks, Food & Wine, Regional & Int...","{""Publisher"": ""Art Post Asia (January 1, 2007)...",9719317051,None,"Paperback – January 1, 2007",None
4448178,Books,Origin of Negative Dialectics,4.9,16,[Susan Buck-Morss examines and stresses the si...,"[About the Author, Susan Buck-Morss is Disting...",18.39,"{'hi_res': [], 'large': [], 'thumb': [], 'vari...","{'title': [], 'url': [], 'user_id': []}",Susan Buck-Morss (Author),"[Books, Politics & Social Sciences, Philosophy]","{""Publisher"": ""Free Press; Trade edition (Dece...",0029051509,None,"Paperback – December 1, 1979",{'avatar': 'https://m.media-amazon.com/images/...
4448179,Books,Trails Illustrated National Parks Guadalupe Mo...,4.8,121,[],[],4.99,"{'hi_res': [], 'large': [], 'thumb': [], 'vari...","{'title': [], 'url': [], 'user_id': []}",None,"[Books, Reference, Atlases & Maps]","{""Language"": ""English"", ""ISBN 10"": ""0925873039...",0

In [5]:
with pd.option_context('display.max_colwidth', None):
    display(metadata_raw_df.sample(10))

main_category  \
873146                Books   
4222608               Books   
464204                Books   
1943638               Books   
4166086               Books   
1501405               Books   
3859886               Books   
2159582               Books   
4072286               Books   
103302   Audible Audiobooks   

                                                                                                             title  \
873146   Shot Down!: When His Bomber Explodes over Nazi-occupied France, Only John Survives-- And Finds a New Life   
4222608      Stephen King 5 Books Collection Box Set (Cujo, 'Salem's Lot, The Shining, Doctor Sleep, Fire Starter)   
464204                                                                                                The Red Tent   
1943638                                        The Thinker's Guide to Clinical Reasoning (Thinker's Guide Library)   
4166086                                                                           The New Negro: An Interpretation   
1501405                                                                                               Raising Ruby   
3859886                                                            Ocean State (Johns Hopkins: Poetry and Fiction)   
2159582                                                                 Antique Trader Perfume Bottles Price Guide   
4072286                                                                                        In Love With Spirit   
103302                                                                       Crescendo: Hush, Hush Trilogy, Book 2   

         average_rating  rating_number  \
873146              4.7              7   
4222608             4.7             94   
464204              4.5          11997   
1943638             4.5             22   
4166086             4.4             53   
1501405             4.5              2   
3859886             5.0              1   
2159582             4.2             41   
4072286             4.3              7   
103302              4.6           2537   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

# Build Feature Transformations

In [6]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [7]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline

In [8]:
# Custom function to reshape data from 1D to 2D
def reshape_2d_to_1d(X):
    return np.array(X).reshape(-1)

def flatten_string_array_col(X):
    return np.array(['\n'.join(x) for x in X])

def tfidf_pipeline_steps():
    steps = [
        ('impute', SimpleImputer(strategy='constant', fill_value='')),
        ('reshape', FunctionTransformer(reshape_2d_to_1d, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=500, ngram_range=(1, 2)))
    ]
    return steps

def description_pipeline_steps():
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=500, ngram_range=(1, 2)))
    ]
    return steps

def categories_pipeline_steps():
    def tokenizer(s):
        return s.split('\n')
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', CountVectorizer(tokenizer=tokenizer, token_pattern=None))
    ]
    return steps

data = pd.Series(["from 14.99", "14.99", "price: 9.99", "20 dollars", "none"])
price_pattern = r'\b((?:\d+\.\d*)|(?:\d+))\b'
display(data.str.extract(price_pattern))

def price_parse_dtype(series, pattern):
    return series.str.extract(pattern).astype(float)

def price_pipeline_steps(price_pattern):
    steps = [
        ('extract_price', FunctionTransformer(price_parse_dtype, kw_args=dict(pattern=price_pattern), validate=False)),
        ('impute', SimpleImputer(strategy='constant', fill_value=0)),
        ('min_max_scale', MinMaxScaler())
    ]
    return steps

,0
0,14.99
1,14.99
2,9.99
3,20
4,NaN


In [9]:
tfm = [
    ('main_category', OneHotEncoder(handle_unknown='ignore'), ['main_category']),  # One-hot encoding for categorical field
    ('title', Pipeline(tfidf_pipeline_steps()), ['title']),  # TF-IDF vectorizer for text field
    ('description', Pipeline(description_pipeline_steps()), 'description'),  # TF-IDF vectorizer for another text field
    ('categories', Pipeline(categories_pipeline_steps()), 'categories'),  # Count Vectorizer for multi-label categorical
    ('price', Pipeline(price_pipeline_steps(price_pattern)), 'price')  # Normalizing price
]
cols = [x[0] for x in tfm]
cols

['main_category', 'title', 'description', 'categories', 'price']

In [10]:
train_full_df = pd.merge(train_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
val_full_df = pd.merge(val_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
train_full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,0920668372,5.0,1430056169000,Books,Love You Forever,"[Review, An extraordinarily different story by...","[Books, Children's Books, Classics]",4.98
1,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,1589255208,5.0,1443926150000,Books,Stories for 2 Year Olds,"[About the Author, Ewa Lipniacka is the author...","[Books, Children's Books, Growing Up & Facts o...",12.89
2,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,2764322836,5.0,1463967052000,Books,Bubble Guppies My Busy Book,[],"[Books, Children's Books]",28.95
3,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,2764330898,5.0,1489085694000,Books,Nickelodeon Paw Patrol My Busy Book -10 Figuri...,"[About the Author, Since 1979, Phidal Publishi...","[Books, Children's Books, Activities, Crafts &...",11.68
4,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,0062380761,5.0,1526591330983,Books,The Girl from Everywhere,"[Review, “History and mythology fans will love...","[Books, Teen & Young Adult, Literature & Fiction]",6.99
...,...,...,...,...,...,...,...,...,...
152582,AFHRNLTISIMZANNK3FV7Y2GHIU4A,1620148021,5.0,1575652999784,Books,Sumo Joe,"[Review, From Booklist:Looking suitably round ...","[Books, Children's Books, Growing Up & Facts o...",20.31
152583,AFHRNLTISIMZANNK3FV7Y2GHIU4A,1546034587,5.0,1620237294827,Books,A Flood of Kindness,"[About the Author, Ellen Leventhal, has been w...","[Books, Children's Books, Growing Up & Facts o...",12.59
152584,AFSYIYI3FQPLRS3P7PTGN6H6FHGA,1940189004,5.0,1412824706000,Books,The River's Memory,"[Review, ""A simply outstanding novel"" --Midwe...","[Books, Literature & Fiction]",2.1
152585,AFSYIYI3FQPLRS3P7PTGN6H6FHGA,1939650380,5.0,1475293391000,Books,Bystanders (SFWP Literary Awards),"[Review, "", Bystanders, is a bold, riveting ma...","[Books, Literature & Fiction, Genre Fiction]",12.0


In [11]:
train_full_df.to_parquet("../data/train_item_features.parquet")
val_full_df.to_parquet("../data/val_item_features.parquet")

# Transformation Pipeline

In [ ]:
# Define preprocessing steps for each column
preprocessing_pipeline = ColumnTransformer(
    transformers=tfm,
    remainder='drop'  # Drop any columns not specified in transformers
)

# Create a pipeline object
item_metadata_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline)
])

# Fit the pipeline
# Drop duplicated item so that the Pipeline only fit the unique item features
fit_df = train_full_df.drop_duplicates(subset=['parent_asin'])
item_metadata_pipeline.fit(fit_df)

# Transform the data (useful for training)
transformed_item_metadata = item_metadata_pipeline.transform(train_full_df)

# For demonstration, print the shape of the transformed features and a few rows
print(f"Transformed Item Metadata Shape: {transformed_item_metadata.shape}")

# Item sequence

# Persist

In [12]:
import joblib
import pickle

In [14]:
joblib.dump(item_metadata_pipeline, '../data/item_metadata_pipeline.joblib')

['../data/item_metadata_pipeline.joblib']